In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RollRig as RollRig

from matplotlib.lines import Line2D

me.mechanics_printing()

In [9]:
half = sym.Rational(1,2)
fwt_panels = 10
main_panels = 20
p = RollRig.base_params(panels = (fwt_panels*2 + main_panels))

In [10]:
# Generate Referecne Frame
wing_frame = mf.HomogenousTransform().R_x(sym.pi+p.q[0])
rhs_fwt_frame = wing_frame.Translate(0,p.s_w*half,0).R_x(-p.q[1])  # RHS Fwt Frame
lhs_fwt_frame = wing_frame.Translate(0,-p.s_w*half,0).R_x(p.q[2])  # LHS Fwt Frame

#Generate Mass Matrices
M_w = ele.MassMatrix(p.m_w)
I_w = ele.MassMatrix(0,I_xx = p.I_xx_w)
fwt_w = ele.MassMatrix(p.m_f,I_xx = p.I_xx_f)

# Generate Rigid Elements
Wing_point_mass = ele.RigidElement(wing_frame.Translate(0,p.y_w,p.z_w),M_w,gravityPotential=True)
wing_inertia = ele.RigidElement(wing_frame,I_w)
rhs_fwt_mass = ele.RigidElement(rhs_fwt_frame.Translate(0,p.l_f,0),fwt_w,gravityPotential=True)
lhs_fwt_mass = ele.RigidElement(lhs_fwt_frame.Translate(0,-p.l_f,0),fwt_w,gravityPotential=True)

In [11]:
sym.simplify(lhs_fwt_frame.Translate(0,-p.l_f,0).Transform_point((0,0,0)))

⎡                 0                  ⎤
⎢                                    ⎥
⎢                   s⋅(σ - 1)⋅cos(q₀)⎥
⎢l_f⋅cos(q₀ + q₂) - ─────────────────⎥
⎢                           2        ⎥
⎢                                    ⎥
⎢                   s⋅(σ - 1)⋅sin(q₀)⎥
⎢l_f⋅sin(q₀ + q₂) - ─────────────────⎥
⎣                           2        ⎦

In [12]:
# Main Wing Aero Forces 
wing_AeroForces = ef.AeroForce_Inverted.PerUnitSpan(p,wing_frame.Translate(0,p.y_i,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = 0,
                               rootAlpha = 0,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = 0,
                               linear = True)#.integrate((p.y_w,-p.s_w*half,p.s_w*half))

In [13]:
# split Main Wing into segments
forces = []
for i in range(main_panels):
    seg_width = p.s_w/main_panels
    yi = -p.s_w/2 + seg_width/2 + i*seg_width
    forces.append(wing_AeroForces.subs({p.y_i:yi,p.a_0:p.a[fwt_panels+i]})*seg_width)
Q = sym.Matrix([0]*p.qs)
for f in forces:
    Q += f.Q()
wing_AeroForces = ef.ExternalForce(Q)

In [14]:
# Left FWT Aero Forces
lhs_fwt_AeroForces = ef.AeroForce_Inverted.PerUnitSpan(p,lhs_fwt_frame.Translate(0,-p.y_f*sym.cos(p.Lambda),0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = 0,
                               rootAlpha = p.alpha_2,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = 0,
                               linear = True)#.integrate((p.y_f,0,p.s_f))

In [15]:
# Left FWT Aero Forces 
rhs_fwt_AeroForces = ef.AeroForce_Inverted.PerUnitSpan(p,rhs_fwt_frame.Translate(0,p.y_f*sym.cos(p.Lambda),0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = 0,
                               rootAlpha = p.alpha_1,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = 0,
                               linear = True)#.integrate((p.y_f,0,p.s_f))

In [16]:
# split FTW's into segments
forces = []
for i in range(fwt_panels):
    seg_width = p.s_f/fwt_panels
    yi = seg_width/2 + i*seg_width
    forces.append(lhs_fwt_AeroForces.subs({p.y_f:yi,p.a_0:p.a[fwt_panels-(i+1)]})*seg_width)
    forces.append(rhs_fwt_AeroForces.subs({p.y_f:yi,p.a_0:p.a[fwt_panels+main_panels+i]})*seg_width)
Q = sym.Matrix([0]*p.qs)
for f in forces:
    Q += f.Q()
fwt_AeroForces = ef.ExternalForce(Q)



In [17]:
Forcing = ef.CustomForce(None) 

CompositeForce = ef.CompositeForce([fwt_AeroForces,wing_AeroForces,Forcing])
# create instance of the model
sm = mf.SymbolicModel.FromElementsAndForces(p,[Wing_point_mass,wing_inertia,rhs_fwt_mass,lhs_fwt_mass],ExtForces = CompositeForce)

In [18]:
sm.to_file('RollRigModel.py')

In [55]:
sym.simplify(sm.M)

⎡                                                                             
⎢                       2                                                     
⎢2⋅I_xxf + I_xxw + 2⋅l_f ⋅m_f - l_f⋅m_f⋅s⋅σ⋅cos(q₁) - l_f⋅m_f⋅s⋅σ⋅cos(q₂) + l_
⎢                                                                             
⎢                                                                             
⎢                                                                 2       l_f⋅
⎢                                                     -I_xxf - l_f ⋅m_f + ────
⎢                                                                             
⎢                                                                             
⎢                                                                 2       l_f⋅
⎢                                                      I_xxf + l_f ⋅m_f - ────
⎣                                                                             

                                           2  2    

In [54]:
sym.simplify(sm.f)

⎡                                                                             
⎢                                                                             
⎢-g⋅l_f⋅m_f⋅cos(q₀ - q₁) + g⋅l_f⋅m_f⋅cos(q₀ + q₂) - g⋅m_w⋅y_w⋅cos(q₀) + g⋅m_w⋅
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [50]:
sm.ExtForces.Q()

⎡                ⎛       s⋅σ⋅cos(Λ)⋅q₂̇   ⎛  s⋅σ⋅cos(Λ)   s⋅σ⋅cos(q₂)   s⋅cos(
⎢                ⎜     - ───────────── + ⎜- ────────── + ─────────── - ───────
⎢   2            ⎜             4         ⎝      4             2            2  
⎢  V ⋅c⋅ρ⋅s⋅σ⋅a₀⋅⎜α₂ + ───────────────────────────────────────────────────────
⎢                ⎝                                   V                        
⎢- ───────────────────────────────────────────────────────────────────────────
⎢                                                               4             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [51]:
y_i = sym.Symbol('y_i')
C_lalphai = sym.Symbol('C_{l{\\alpha} i}')
b_i = sym.Symbol('b_i')
Q = wing_AeroForces.subs({p.y_i:y_i,p.a_0:C_lalphai})*b_i
Q.Q()

⎡                            2    ⎤
⎢-V⋅c⋅ρ⋅C_{l{\alpha} i}⋅bᵢ⋅yᵢ ⋅q₀̇ ⎥
⎢─────────────────────────────────⎥
⎢                2                ⎥
⎢                                 ⎥
⎢                0                ⎥
⎢                                 ⎥
⎣                0                ⎦

In [52]:
Q = lhs_fwt_AeroForces.subs({p.y_f:y_i,p.a_0:C_lalphai})*b_i
sym.simplify(Q.Q())

⎡V⋅c⋅ρ⋅C_{l{\alpha} i}⋅bᵢ⋅(2⋅V⋅α₂ - 2⋅yᵢ⋅cos(Λ)⋅q₂̇ - (-s⋅σ⋅cos(q₂) + s⋅cos(q₂
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                    8        
⎢                                                                             
⎢                                                                    0        
⎢                                                                             
⎢               V⋅c⋅ρ⋅C_{l{\alpha} i}⋅bᵢ⋅yᵢ⋅(2⋅V⋅α₂ - 2⋅yᵢ⋅cos(Λ)⋅q₂̇ - (-s⋅σ⋅
⎢               ──────────────────────────────────────────────────────────────
⎣                                                                   4         

) + 2⋅yᵢ⋅cos(Λ))⋅q₀̇)⋅(-s⋅σ⋅cos(q₂) + s⋅cos(q₂) + 2⋅yᵢ⋅cos(Λ))⎤
────────────────────────────────────────────────────────────⎥
                                                            ⎥
                                                            ⎥
                                      